In [1]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current directory
print("Current directory:", current_directory)

Current directory: /my_bot/src/my_robot_controller/my_robot_controller


## GPT-4o

In [1]:
import sys
print(sys.executable)

/bin/python3


In [3]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 KB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 KB 4.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 KB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 KB 18.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
source /opt/ros/humble/setup.bash


NameError: name 'source' is not defined

In [ ]:
ros2 run rqt_image_view rqt_image_view

In [7]:
from openai import OpenAI
import base64
import requests
from PIL import Image
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np

In [9]:
with open('/my_bot/src/my_robot_controller/resource/config.txt', 'r') as file:
    api_key = file.read().strip()
    
def GPT4o(robotPrompt, imgDecode, api_key = api_key):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": robotPrompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{imgDecode}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }
    return requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload).json()['choices'][0]['message']['content']

### Prompt

In [10]:
delimiter = "#####"

In [11]:
task = """Given the robot view, select the best action for a mobile robot to move to a football goal post. Stop if the robot reaches very close to the goal post.
"""

actionInstructions = """
Action Selection Instruction 1: Move forward. (Action = 1)
Action Selection Instruction 2: Move right. (Action = 2).
Action Selection Instruction 3: Move left. (Action = 3).
Action Selection Instruction 4: Stop. (Action = 4).
"""

outputInstructions = f"""
Your response should use the following format:
<reasoning>
<reasoning>
<repeat until you have a decision>
Response to user:{delimiter} <only output one `Action_id` as a int number of you decision, without any action name or explanation> 
Make sure to include {delimiter} to separate every step."""

robotPrompt = f"""
{delimiter} Task:
{task}
{delimiter} Action Selection Instructions:
{actionInstructions}
{delimiter} Output Instructions:
{outputInstructions}
"""

In [12]:
print(robotPrompt)


##### Task:
Given the robot view, select the best action for a mobile robot to move to a football goal post. Stop if the robot reaches very close to the goal post.

##### Action Selection Instructions:

Action Selection Instruction 1: Move forward. (Action = 1)
Action Selection Instruction 2: Move right. (Action = 2).
Action Selection Instruction 3: Move left. (Action = 3).
Action Selection Instruction 4: Stop. (Action = 4).

##### Output Instructions:

Your response should use the following format:
<reasoning>
<reasoning>
<repeat until you have a decision>
Response to user:##### <only output one `Action_id` as a int number of you decision, without any action name or explanation> 
Make sure to include ##### to separate every step.



### Sensor Reading

In [13]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

img_path_front = '/my_bot/src/my_robot_controller/resource/temp/goalPostFront.png'
post_front = encode_image(img_path_front)

img_path_right = '/my_bot/src/my_robot_controller/resource/temp/goalPostRight.png'
post_right = encode_image(img_path_right)

img_path_near = '/my_bot/src/my_robot_controller/resource/temp/goalPostNear.png'
post_near = encode_image(img_path_near)

### Reasoning

In [14]:
response = GPT4o(robotPrompt, post_near)

In [15]:
print(response)

##### The robot is very close to the goal post.
##### Action 4 is to stop if the robot is very close to the goal post.
Response to user:##### 4


In [16]:
import re

pattern = r"#####\s*(\d+)"

# Search for the pattern in the response text
match = re.search(pattern, response)

if match:
    # Extract the integer value
    int_value = int(match.group(1))
    print(f"Extracted integer value: {int_value}")
else:
    print("No match found.")

Extracted integer value: 4
